## Model_Ensamble with crab age dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
crab_df=pd.read_csv('train.csv',index_col=0)

In [3]:
lbl=LabelEncoder()

In [4]:
crab_df=pd.get_dummies(crab_df,drop_first=True)
crab_df

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,Sex_I,Sex_M
id,,,,,,,,,,
0,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9,True,False
1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8,True,False
2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9,False,True
3,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11,False,False
4,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8,True,False
...,...,...,...,...,...,...,...,...,...,...
74046,1.6625,1.2625,0.4375,50.660556,20.680960,10.361742,12.332033,10,False,False
74047,1.0750,0.8625,0.2750,10.446791,4.323299,2.296310,3.543687,6,True,False
74048,1.4875,1.2000,0.4125,29.483480,12.303683,7.540967,8.079607,10,False,False


In [5]:
X=crab_df.drop('Age',axis=1)
y=crab_df['Age']

In [6]:
elastic=ElasticNet()

In [7]:
scaler=StandardScaler()

In [8]:
knn=KNeighborsRegressor()

In [9]:
dtr=DecisionTreeRegressor()

In [10]:
from sklearn.pipeline import Pipeline

In [11]:
pipe=Pipeline([('SCL',scaler),('KNN',knn)])

In [12]:
ensamble=VotingRegressor(estimators=[('Elastic',elastic),('KNN',pipe),('DTR',dtr)],verbose=3)

In [13]:
ensamble.get_params()

{'estimators': [('Elastic', ElasticNet()),
  ('KNN',
   Pipeline(steps=[('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())])),
  ('DTR', DecisionTreeRegressor())],
 'n_jobs': None,
 'verbose': 3,
 'weights': None,
 'Elastic': ElasticNet(),
 'KNN': Pipeline(steps=[('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())]),
 'DTR': DecisionTreeRegressor(),
 'Elastic__alpha': 1.0,
 'Elastic__copy_X': True,
 'Elastic__fit_intercept': True,
 'Elastic__l1_ratio': 0.5,
 'Elastic__max_iter': 1000,
 'Elastic__positive': False,
 'Elastic__precompute': False,
 'Elastic__random_state': None,
 'Elastic__selection': 'cyclic',
 'Elastic__tol': 0.0001,
 'Elastic__warm_start': False,
 'KNN__memory': None,
 'KNN__steps': [('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())],
 'KNN__verbose': False,
 'KNN__SCL': StandardScaler(),
 'KNN__KNN': KNeighborsRegressor(),
 'KNN__SCL__copy': True,
 'KNN__SCL__with_mean': True,
 'KNN__SCL__with_std': True,
 'KNN__KNN__algorithm': 'auto',
 'KNN__KNN_

In [14]:
params={'Elastic__alpha':np.linspace(0.001,5,5),'Elastic__l1_ratio':np.linspace(0,1,5),'KNN__SCL':[StandardScaler(),MinMaxScaler()],'KNN__KNN__n_neighbors':[1,2,3,4],'DTR__max_depth':[None,3,5],'DTR__min_samples_split':[2,5,10],'DTR__min_samples_leaf':[1,5,10]}

In [15]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [16]:
from sklearn.model_selection import StratifiedKFold

In [17]:
kfolf=StratifiedKFold(n_splits=5, shuffle=True, random_state=23)

In [18]:
rgcv=RandomizedSearchCV(ensamble,param_distributions=params,cv=kfolf,scoring='r2')

In [19]:
rgcv.fit(X,y)

[Voting] .................. (1 of 3) Processing Elastic, total=   0.1s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.1s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.1s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.1s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.1s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Votin

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.789e+05, tolerance: 5.970e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[Voting] .................. (1 of 3) Processing Elastic, total=   0.3s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.5s


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.788e+05, tolerance: 5.974e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[Voting] .................. (1 of 3) Processing Elastic, total=   0.3s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.5s


...

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.739e+05, tolerance: 5.972e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[Voting] .................. (1 of 3) Processing Elastic, total=   0.3s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.1s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.3s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.3s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.3s
[Voting] .................. (1 of 3) Processing Elastic, total=   0.0s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.1s
[Votin

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.171e+05, tolerance: 7.466e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[Voting] .................. (1 of 3) Processing Elastic, total=   0.3s
[Voting] ...................... (2 of 3) Processing KNN, total=   0.2s
[Voting] ...................... (3 of 3) Processing DTR, total=   0.1s


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
                   estimator=VotingRegressor(estimators=[('Elastic',
                                                          ElasticNet()),
                                                         ('KNN',
                                                          Pipeline(steps=[('SCL',
                                                                           StandardScaler()),
                                                                          ('KNN',
                                                                           KNeighborsRegressor())])),
                                                         ('DTR',
                                                          DecisionTreeRegressor())],
                                             verbose=3),
                   param_distributions={'DTR__max_depth': [None, 3, 5],
                                        'DTR__min_samples_leaf': [1, 5, 10],
                                        'DTR__min_samples_split': [2, 5, 10],
                                        'Elastic__alpha': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                                        'Elastic__l1_ratio': array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
                                        'KNN__KNN__n_neighbors': [1, 2, 3, 4],
                                        'KNN__SCL': [StandardScaler(),
                                                     MinMaxScaler()]},
                   scoring='r2')

In [20]:
rgcv.best_score_

0.5413108442694662

In [21]:
rgcv.best_params_

{'KNN__SCL': StandardScaler(),
 'KNN__KNN__n_neighbors': 3,
 'Elastic__l1_ratio': 0.0,
 'Elastic__alpha': 2.5004999999999997,
 'DTR__min_samples_split': 5,
 'DTR__min_samples_leaf': 5,
 'DTR__max_depth': 5}

In [44]:
rgcv.best_estimator_

VotingRegressor(estimators=[('Elastic',
                             ElasticNet(alpha=2.5004999999999997,
                                        l1_ratio=0.0)),
                            ('KNN',
                             Pipeline(steps=[('SCL', StandardScaler()),
                                             ('KNN',
                                              KNeighborsRegressor(n_neighbors=3))])),
                            ('DTR',
                             DecisionTreeRegressor(max_depth=5,
                                                   min_samples_leaf=5,
                                                   min_samples_split=5))],
                verbose=3)

In [31]:
test_new_df=pd.read_csv('test.csv',index_col=0)

In [33]:
test_new_df=pd.get_dummies(test_new_df,drop_first=True)

In [34]:
test_new_df

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Sex_I,Sex_M
id,,,,,,,,,
74051,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552,True,False
74052,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930,True,False
74053,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415,False,False
74054,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676,False,False
74055,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066,True,False
...,...,...,...,...,...,...,...,...,...
123414,1.3000,1.0375,0.3250,16.315137,6.690482,5.173784,3.756309,False,False
123415,1.0375,0.7625,0.2625,10.276694,4.436697,1.998640,3.543687,True,False
123416,1.4875,1.1625,0.3625,31.382897,11.396499,6.846404,8.788345,False,False


In [35]:
y_pred=rgcv.predict(test_new_df)

In [36]:
y_pred

array([ 7.44161235,  7.81984293, 10.19037008, ..., 13.23494095,
        9.66514767, 11.92062678])

In [39]:
sample_submission_df=pd.read_csv('sample_submission.csv',index_col=0)

In [40]:
sample_submission_df

,Age
id,
74051,10
74052,10
74053,10
74054,10
74055,10
...,...
123414,10
123415,10
123416,10


In [41]:
sample_submission_df['Age']=y_pred

In [42]:
sample_submission_df

,Age
id,
74051,7.441612
74052,7.819843
74053,10.190370
74054,10.185395
74055,7.750905
...,...
123414,8.640982
123415,8.351339
123416,13.234941


In [43]:
sample_submission_df.to_csv('sample_submission.csv')